In [4]:
from tensorflow import keras
model = keras.models.load_model('FinalModel.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          5000000   
_________________________________________________________________
lstm (LSTM)                  (None, 125)               113000    
_________________________________________________________________
dense (Dense)                (None, 2)                 252       
Total params: 5,113,252
Trainable params: 5,113,252
Non-trainable params: 0
_________________________________________________________________


In [14]:
import pandas as pd

In [17]:
df = pd.read_csv('MainData.csv')
df = df.dropna()

In [19]:
import re
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from spellchecker import SpellChecker
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['cleanedText'].values)

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
    words = [word for word in text.split() if word.lower() not in sw_spacy]
    text = " ".join(words)
    return text

def fuzzyLogic(text):
  spell = SpellChecker()
  text = ' '.join([spell.correction(word) for word in text.split()])
  return text

def isDepressing(text):
  text = fuzzyLogic(text)
  lst = {'body':[text]}
  # Calling DataFrame constructor on list  
  dframe = pd.DataFrame(lst) 
  dframe = dframe['body'].apply(clean_text)
  dframe = dframe.str.replace('\d+', '')
  X1 = tokenizer.texts_to_sequences(dframe.values)
  X1 = pad_sequences(X1, maxlen=MAX_SEQUENCE_LENGTH)
  print('Shape of data tensor:', X1.shape)
  a = model.predict(X1)
  print(a)
  if(a[0][0] > a[0][1]):
    return 0
  return 1

In [23]:
isDepressing("i get depressing thoughts every day. i plan on committing suicide")

Shape of data tensor: (1, 250)
[[0.00859726 0.9914028 ]]


/var/folders/5d/7kyh57w16sx0z4b2sprf9ln00000gn/T/ipykernel_74992/552715922.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
  dframe = dframe.str.replace('\d+', '')


1